In [30]:
import numpy as np

In [26]:
def dameraulevenshtein(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition
    return d[lenstr1-1,lenstr2-1] 

In [43]:
def megazord_metric(input_counters_1, input_counters_2, n, threshold, coefficient):
  '''
    Gets lists of n-grams, sorted by rank, threshold for distinction of their frequency differences,
    coefficient of quantitative difference, n of n-grams. 
    Returns void.
    Params: 
      input_counters_1: a list of tuples in form (n-gram, rank) for lect 1.
      input_counters_2: a list of tuples in form (n-gram, rank) for lect 2.
      n: n in n-grams
      threshold: threshold for frequency difference distinction.
      coefficient: multiplying factor, based on how different quantitatively counters are.
    Returns:
      void
  '''
  in_l1_not_in_l2 = [i for i in input_counters_1 if i[0] not in [j[0] for j in input_counters_2]]
  in_l2_not_in_l1 = [i for i in input_counters_2 if i[0] not in [j[0] for j in input_counters_1]]
  aa = [n for m in [in_l2_not_in_l1, in_l1_not_in_l2] for n in m]
  common_l1 = [i for i in input_counters_1 if i not in in_l1_not_in_l2]
  common_l2 = [i for i in input_counters_2 if i not in in_l2_not_in_l1]
  AA = []
  Aa = []
  for i in common_l1:
    max_lev = n + 1
    dist_rank = 0
    for j in common_l2:
      lev = dameraulevenshtein(i[0], j[0])
      if (max_lev > lev):
        max_lev = lev
        dist_rank = abs(i[1] - j[1])
    dist_rank_lev = dist_rank + max_lev
    if (dist_rank_lev <= threshold):
      AA.append((i[0], dist_rank_lev))
      continue
    Aa.append((i[0], dist_rank_lev))
  overall_length = len(input_counters_1) + len(input_counters_2)
  AA_rel = len(AA)*2/overall_length
  Aa_rel = len(Aa)*2/overall_length
  aa_rel = len(aa)/overall_length
  A = AA_rel + 0.5 * Aa_rel
  a = aa_rel + 0.5 * Aa_rel 
  equilibrium = 2 * A * a
  Fst = (equilibrium - Aa_rel)/equilibrium
  return Fst

In [44]:
a = [("a", 1), ("ab", 2), ("aa", 3), ("b", 4)]
b = [("a", 1), ("aa", 2), ("c", 3)]
F = megazord_metric(a, b, 3, 0, 1)
print(F)

0.41666666666666663
